In [1]:
export WORK=~/Projects/app-master

# Cloud Application Deployment Walk-Through


## Priprava infrastruktury na nasadenie aplikacie
- Container Registry
- Storage Account


In [2]:
cd ${WORK}/app-terraform/init

terraform init




Initializing provider plugins...

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


In [3]:
terraform plan





Refreshing Terraform state in-memory prior to plan...
The refreshed state will be used to calculate this plan, but will not be
persisted to local or remote state storage.

azurerm_resource_group.resource_group: Refreshing state... (ID: /subscriptions/3f717624-c999-4be3-8245-36205977d2ce/resourceGroups/nc19-dev)
azurerm_container_registry.nc19cr: Refreshing state... (ID: /subscriptions/3f717624-c999-4be3-8245-...ft.ContainerRegistry/registries/nc19cr)
azurerm_storage_account.storage_account: Refreshing state... (ID: /subscriptions/3f717624-c999-4be3-8245-...crosoft.Storage/storageAccounts/nc19sa)
azurerm_storage_container.terraform_storage_container: Refreshing state... (ID: https://nc19sa.blob.core.windows.net/terraform-storage)

------------------------------------------------------------------------

An execution plan has been generated and is shown below.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  + azu

In [4]:
terraform apply -auto-approve





azurerm_resource_group.resource_group: Refreshing state... (ID: /subscriptions/3f717624-c999-4be3-8245-36205977d2ce/resourceGroups/nc19-dev)
azurerm_container_registry.nc19cr: Refreshing state... (ID: /subscriptions/3f717624-c999-4be3-8245-...ft.ContainerRegistry/registries/nc19cr)
azurerm_storage_account.storage_account: Refreshing state... (ID: /subscriptions/3f717624-c999-4be3-8245-...crosoft.Storage/storageAccounts/nc19sa)
azurerm_storage_container.terraform_storage_container: Refreshing state... (ID: https://nc19sa.blob.core.windows.net/terraform-storage)
azurerm_resource_group.resource_group: Creating...
  location: "" => "westeurope"
  name:     "" => "nc19-dev"
  tags.%:   "" => "<computed>"
azurerm_resource_group.resource_group: Creation complete after 1s (ID: /subscriptions/3f717624-c999-4be3-8245-36205977d2ce/resourceGroups/nc19-dev)
azurerm_storage_account.storage_account: Creating...
  access_tier:                       "" => "<computed>"
  account_encryption_source:  

In [6]:
export DOCKER_PASSWORD=$(terraform output ContainerRegistryPassword)

# Dotnet Core Application Build

In [8]:
cd ${WORK}/app-dotnet

docker build -t nc19cr.azurecr.io/nc19app:1 .


Sending build context to Docker daemon  46.36MB
Step 1/10 : FROM mcr.microsoft.com/dotnet/core/sdk:3.0 AS build-env
 ---> 4422e7fb740c
Step 2/10 : WORKDIR /app
 ---> Using cache
 ---> 7f401fdd64f1
Step 3/10 : COPY *.csproj ./
 ---> Using cache
 ---> f04c0bb3369f
Step 4/10 : RUN dotnet restore
 ---> Using cache
 ---> e8db47ae5b9f
Step 5/10 : COPY . ./
 ---> Using cache
 ---> 3478882ffa5e
Step 6/10 : RUN dotnet publish -c Release -o out
 ---> Using cache
 ---> 4d5bb4f7adde
Step 7/10 : FROM mcr.microsoft.com/dotnet/core/aspnet:3.0-alpine
 ---> 1cb704cc94a0
Step 8/10 : WORKDIR /app
 ---> Using cache
 ---> c91120c9c779
Step 9/10 : COPY --from=build-env /app/out .
 ---> Using cache
 ---> a75e0d3e8744
Step 10/10 : ENTRYPOINT ["dotnet", "ContosoUniversity.dll"]
 ---> Using cache
 ---> a48198da3e69
Successfully built a48198da3e69
Successfully tagged nc19cr.azurecr.io/nc19app:1


In [9]:
docker login nc19cr.azurecr.io -u nc19cr -p ${DOCKER_PASSWORD}


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Login Succeeded


In [10]:
docker push nc19cr.azurecr.io/nc19app:1

The push refers to repository [nc19cr.azurecr.io/nc19app]

c3a49aad: Preparing 
3636a9a6: Preparing 
9cbc2153: Preparing 
7d650ebc: Preparing 
4b3d1f37: Preparing 
1: digest: sha256:a919d50bbec44e7abd07964e337545805213d2596c8d97eb27be9f877d81046a size: 1580


# Set-up of infrastructure and deploy application using terraform

In [15]:
cd ${WORK}/app-terraform/app-infra

terraform init


Initializing provider plugins...

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


In [16]:
terraform plan \
 -var docker_registry=nc19cr \
 -var docker_repository=nc19app \
 -var docker_tag=1 \
 -var docker_password=${DOCKER_PASSWORD} 

Refreshing Terraform state in-memory prior to plan...
The refreshed state will be used to calculate this plan, but will not be
persisted to local or remote state storage.

azurerm_resource_group.resource_group: Refreshing state... (ID: /subscriptions/3f717624-c999-4be3-8245-36205977d2ce/resourceGroups/dev-nc19app)
azurerm_app_service_plan.app_service_plan: Refreshing state... (ID: /subscriptions/3f717624-c999-4be3-8245-...osoft.Web/serverfarms/app_service_plan)
azurerm_app_service.app_service: Refreshing state... (ID: /subscriptions/3f717624-c999-4be3-8245-...rs/Microsoft.Web/sites/nc19-appservice)

------------------------------------------------------------------------

An execution plan has been generated and is shown below.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  + azurerm_app_service.app_service
      id:                                               <computed>
      app_service_plan_id:               

In [17]:
terraform apply -auto-approve \
 -var docker_registry=nc19cr \
 -var docker_repository=nc19app \
 -var docker_tag=1 \
 -var docker_password=${DOCKER_PASSWORD} 

azurerm_resource_group.resource_group: Refreshing state... (ID: /subscriptions/3f717624-c999-4be3-8245-36205977d2ce/resourceGroups/dev-nc19app)
azurerm_app_service_plan.app_service_plan: Refreshing state... (ID: /subscriptions/3f717624-c999-4be3-8245-...osoft.Web/serverfarms/app_service_plan)
azurerm_app_service.app_service: Refreshing state... (ID: /subscriptions/3f717624-c999-4be3-8245-...rs/Microsoft.Web/sites/nc19-appservice)
azurerm_resource_group.resource_group: Creating...
  location:         "" => "westeurope"
  name:             "" => "dev-nc19app"
  tags.%:           "" => "1"
  tags.environment: "" => "dev"
azurerm_resource_group.resource_group: Creation complete after 1s (ID: /subscriptions/3f717624-c999-4be3-8245-36205977d2ce/resourceGroups/dev-nc19app)
azurerm_app_service_plan.app_service_plan: Creating...
  app_service_environment_id:   "" => "<computed>"
  kind:                         "" => "Linux"
  location:                     "" => "westeurope"
  maximum_elastic_wo